<a href="https://colab.research.google.com/github/Samir-atra/Expenses_tracker/blob/main/project/utils/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports

import tensorflow as tf
import pathlib
import IPython
import numpy as np

2023-10-19 21:47:02.450617: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-19 21:47:02.601111: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-19 21:47:02.601138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-19 21:47:02.601753: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-19 21:47:02.668702: I tensorflow/core/platform/cpu_feature_g

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
# load the data

data_dir = pathlib.Path('/home/samer/Desktop/Tobacco3482-jpg/')


dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
    data_dir,
    labels= 'inferred',
    # validation_split=0.2,
    # subset='training',
    seed= 1,
    batch_size=5,
    image_size=(300,300),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

# dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
#     data_path,
#     labels= 'inferred',
#     validation_split=0.2,
#     subset='validation',
#     seed= 2,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

Found 4368 files belonging to 12 classes.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [3]:
# download the transfer learning pre-trained model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (300, 300, 3),
    include_top = False,
    weights = "imagenet"
)

In [4]:
# save the pre-trained model
saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/Expenses_tracker_stuff/expenses_tracker_classifier95%test.h5')

base_model.save(saving_path)

/home/samer/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
# load the pre-trained model, build the training model and train it.

base_model = tf.keras.models.load_model(saving_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1)
])


inputs = tf.keras.Input(shape=(300, 300, 3))
# x = augmentation(inputs)
x = rescaling(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(12, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

checkpoint_filepath = '/home/samer/Desktop/Beedoo/Expenses_tracker_stuff/checkpoint/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=20,
    callbacks=[model_checkpoint_callback])


# model.load_weights(checkpoint_filepath)
# model.evaluate(
#     x=x_test,
#     y=y_test,)

full_model_saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/Expenses_tracker_stuff/expenses_tracker_model.h5')

tf.keras.models.save_model(model,                                     # saving the fully trained model
                           full_model_saving_path,
                           overwrite=True,
                           save_format='tf'
                           )

Epoch 1/20


2023-10-18 17:27:35.094331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-10-18 17:27:39.287680: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.85GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-18 17:27:39.287750: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.64GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-18 17:27:40.053089: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.07GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

209/875 [======>.......................] - ETA: 1:33 - loss: 1.6420 - accuracy: 0.4727

2023-10-18 17:28:10.384975: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


801/875 [==========================>...] - ETA: 10s - loss: 1.1664 - accuracy: 0.6155

...

875/875 [==============================] - 155s 177ms/step - loss: 0.3794 - accuracy: 0.8715
Epoch 20/20
208/875 [======>.......................] - ETA: 2:15 - loss: 0.3810 - accuracy: 0.8731

2023-10-18 18:16:17.826308: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


801/875 [==========================>...] - ETA: 14s - loss: 0.3652 - accuracy: 0.8794

2023-10-18 18:18:13.821199: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


875/875 [==============================] - 173s 198ms/step - loss: 0.3664 - accuracy: 0.8795
